In [ ]:




def main_program_full( input_filename ):

    
    with open( input_filename, 'r' ) as stream:
        input_dict = yaml.load(stream, Loader=yaml.SafeLoader)

    # control parameters
    filename          = input_dict['control']['name']
    sigma_rescale     = bool(input_dict['control']['laser']['sigma_rescale'])
    sample_electrons_total  = int(float( input_dict['control']['sample_electrons'] ))
    sample_batch_size = int(1e7)

    n_samp = sample_electrons_total // sample_batch_size
    r_samp = sample_electrons_total % sample_batch_size

    if r_samp:
        sampling_batches = [sample_batch_size]*int(n_samp) + [r_samp]
    else:
        sampling_batches = [sample_batch_size]*int(n_samp)

    print (sampling_batches,sum(sampling_batches),sample_electrons_total)






    # laser parameters
    omega0 = float( input_dict['laser']['omega0']  )
    a0     = float( input_dict['laser']['a0']      )
    Tpulse = float( input_dict['laser']['Tpulse']  )
    pol    = float( input_dict['laser']['pol']     )
    w0     = float( input_dict['laser']['w0']      )
    pulse  = input_dict['laser']['pulse']

    if pulse!='cos2':
        raise NotImplementedError

    # translate Tpulse==FWHM in fs --> sigma parameter which is dimensionless
    sigma = 2.0852201339 * Tpulse * omega0

    if sigma_rescale:
        sigma_crit =  float( input_dict['control']['laser']['sigma_crit']  )
        if sigma > sigma_crit:
            sigma_rescalefactor = sigma / sigma_crit
            print (f' >> rescale pulse duration: {sigma:.2f} -> {sigma_crit:.2f}: sigma_rescalefactor = {sigma_rescalefactor:.2f}')
        else:
            sigma_rescalefactor = 1.
    else:
        sigma_crit          = 0
        sigma_rescalefactor = 1.


##########################################################################################################
    # extract beam parameters
    gamma0       = float( input_dict['beam']['gamma'] )
    energyspread = float( input_dict['beam']['energyspread'] )
    emittance    = float( input_dict['beam']['emittance'] )
    beam_size_X    = float( input_dict['beam']['sigmaX'] )        # transverse beam size x axis in microns
    beam_size_Y    = float( input_dict['beam']['sigmaY'] )        # transverse beam size y axis in microns
    beam_length  = float( input_dict['beam']['sigmaL'] )        # longitudinal beam size in microns 
    beam_charge  = float( input_dict['beam']['beam_charge'])



    rms_angle_X   = emittance / beam_size_X / gamma0
    
    rms_angle_Y   = emittance / beam_size_Y / gamma0
############################################################################################################

    # extract detector parameters
    omega_detector = [float(w) for w in input_dict['detector']['omega']]
    theta_detector = [float(t) for t in input_dict['detector']['theta']]



    number_electrons = int( beam_charge / 1.60217653e-19)

    # sample_electrons = int(5e7) # number of elecrons used for MC sampling, determines weight of each emitted photon
    electron_weight  = number_electrons / sample_electrons_total
    print (f' >> number_electrons   = {number_electrons}')
    print (f' >> sample_electrons   = {sample_electrons_total}')
    print (f' >> electron weight    = {electron_weight:.4g}')


    keep_xi_peak   = []
    X_photon       = []
    K_photon       = []
    W_photon       = []

    X_electron     = []
    P_electron     = []
    W_electron     = []


    print (' >> MC sampling')

    for jj,sample_electrons in enumerate(sampling_batches):

        print (f'  > batch {jj} : {sample_electrons} macroelectrons')

        # print (len(W_photon))

        '''
        theta_x      = np.random.normal( 0.     , rms_angle           , sample_electrons )
        theta_y      = np.random.normal( 0.     , rms_angle           , sample_electrons )

        x            = np.random.normal( 0 , beam_size    , sample_electrons )      
        y            = np.random.normal( 0 , beam_size    , sample_electrons )

        '''
        
        
        mean_x = [0,0]  # x-offset and x' offset for focus 
        cov_x = Covariant_Matrix(beam_size_X,rms_angle_X)
        
        mean_y = [0,0]  # y-offset and y' offset for focus
        cov_y = Covariant_Matrix(beam_size_Y,rms_angle_Y)
        
        
        x,theta_x = np.random.multivariate_normal(mean_x, cov_x, sample_electrons).T
        y,theta_y = np.random.multivariate_normal(mean_y, cov_y, sample_electrons).T
        
        
        gamma        = np.random.normal( gamma0 , gamma0*energyspread , sample_electrons )
        beta         = sqrt(1-1./gamma**2)
              
        r            = np.sqrt( x**2 + y**2 )

        xi_peak      = a0 * exp( -r**2/w0**2 )


        omega        = np.random.uniform(omega_detector[0], omega_detector[1], sample_electrons) 
        theta        = np.random.uniform(theta_detector[0], theta_detector[1], sample_electrons) 
        phi          = np.random.uniform(0,2*pi,sample_electrons)


        pz0          = gamma*beta*cos(theta_x)*cos(theta_y)
        px0          = gamma*beta*sin(theta_x)
        py0          = gamma*beta*sin(theta_y)
        pt0          = sqrt( 1 + px0**2 + py0**2 + pz0**2 )

 
        U_in         = asarray([pt0,px0,py0,pz0])


        spec_weight  =  sigma_rescalefactor


        # print (gamma.shape,theta_x.shape,U_in.shape,)
        rad_int  = radiation_spectrum_linear( U_in , xi_peak , omega0 , sigma / sigma_rescalefactor , omega, theta, phi )

        spectrum = spec_weight * rad_int * sin(theta)

        spec_max = amax(spectrum)



        s_val        = np.random.uniform(0             , spec_max,       sample_electrons)


        samplingvolume     = spec_max * (omega_detector[1]-omega_detector[0]) * (theta_detector[1]-theta_detector[0])
        base_photon_weight = 2*pi * samplingvolume * electron_weight

        selector1          = s_val < spectrum
        number_photons     = sum(selector1)
        print ('   base photon weight :' , base_photon_weight)
        print ('   number photons     :' , number_photons)



        keep_gamma   = gamma[selector1]
        keep_theta_x = theta_x[selector1]
        keep_theta_y = theta_y[selector1]

        keep_omega   = omega[selector1]
        keep_theta   = theta[selector1]
        keep_phi     = phi[selector1]




        keep_x       = x[selector1]
        keep_y       = y[selector1]
        keep_zeta    = np.random.normal( 0 , beam_length  , number_photons )

        keep_z       = +0.5 * keep_zeta
        keep_t       = -0.5 * keep_zeta


        K0           = keep_omega
        K1           = keep_omega * sin(keep_theta) * cos(keep_phi)
        K2           = keep_omega * sin(keep_theta) * sin(keep_phi)
        K3           = keep_omega * cos(keep_theta) 



        keep_xi_peak.extend(xi_peak[selector1])

        X_photon.extend(    np.asarray( (keep_t,keep_x,keep_y,keep_z) ).T)
        K_photon.extend(    np.asarray( (K0,K1,K2,K3) ).T                )
        W_photon.extend(    base_photon_weight * ones(number_photons)    )


        # electrons that have emitted
        px_out  = px0[selector1] * elec_mass - K1
        py_out  = py0[selector1] * elec_mass - K2
        pz_out  = pz0[selector1] * elec_mass - K3 
        pt_out  = sqrt( elec_mass**2 + px_out**2 + py_out**2 + pz_out**2 )


        X_electron.extend( np.asarray( (keep_t,keep_x,keep_y,keep_z) ).T )
        P_electron.extend( np.asarray( (pt_out,px_out,py_out,pz_out) ).T )
        W_electron.extend( base_photon_weight * ones(number_photons)     )

        print ('   total photon number:',len(W_electron))


    # convert lists into ndarrays
    X_photon     = np.asarray(X_photon)
    K_photon     = np.asarray(K_photon)
    W_photon     = np.asarray(W_photon)
    keep_xi_peak = np.asarray(keep_xi_peak)

    X_electron   = np.asarray(X_electron)
    P_electron   = np.asarray(P_electron)
    W_electron   = np.asarray(W_electron)


    return X_photon,K_photon,W_photon,X_electron,P_electron,W_electron,keep_xi_peak

